In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from copy import copy

from nn_backend.import_xyz_img import import_xyz_img
from nn_backend.conv_imgs import conv_imgs
from sklearn.metrics import r2_score, mean_absolute_error
from matplotlib import cm, colormaps
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import ShuffleSplit
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, concatenate
from keras.optimizers import Adam
from keras.utils import set_random_seed
from scipy.stats import norm
import seaborn as sns
import math
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from scipy import ndimage
from nn_backend.modified_loss_functions import cube_msle, linear_msle
from skimage import morphology
from nn_backend.data_import import import_curves_from_folder, curve_calibration, matrix_cut, data_augment, mask_from_topo
from nn_backend.nn_for_ep import nn_for_ep
from nn_backend.post_process import prediction_reconstruction_with_train, img_export, histogram_export



viridis = colormaps['viridis']
newcolors = viridis(np.linspace(0, 1, 256))
black = np.array([0, 0, 0, 1])
newcolors[0:33, :] = black
newcmp = ListedColormap(newcolors)
del black, newcolors, viridis

In [52]:
new_curves_folder=rf'c:\Users\mcano\Code\rawdata\img_data\FHRbact1\Out_GetEpOnpx_2024-4_from_OUT_qi_sh4_tip2-data-2023.11.10-16.34.43.387_21-Mar-2024-19-13-48\tablas'

In [50]:
a=pd.read_table(rf'c:\Users\mcano\Code\rawdata\img_data\FHRbact1\Out_GetEpOnpx_2024-4_from_OUT_qi_sh4_tip2-data-2023.11.10-16.34.43.387_21-Mar-2024-19-13-48\tablas\Tabla_I_66_J_100.txt', skiprows=19)
a=a.pivot_table(columns='Z[nm]', values='dCdz[aF/nm]', index='ep[none]')
zs=np.array(a.columns)
names=['ep']
for i in zs:
    names.append(rf'Z={i} nm')
eps=np.array(a.index)
t=np.column_stack((eps,np.array(a)))
out=pd.DataFrame(t, columns=names) 

In [53]:
onlyfiles = [f for f in listdir(new_curves_folder) if isfile(join(new_curves_folder, f))]


In [71]:
k=0
idxs=np.zeros((2,1))
for k in np.arange(len(onlyfiles)):
    files=onlyfiles[k]
    files=files.replace("Tabla_I_", "")
    files=files.replace("_J_", "_")
    files=files.replace(".txt", "")
    files=np.array(files.split("_")).astype(int)
    files=np.expand_dims(files, axis=1)

    idxs=np.concatenate((idxs,files), axis=1)
idxs=idxs[:,1:]